In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score , roc_curve
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier 
import gc

import optuna


In [ ]:
pip install datatable

In [ ]:
import datatable as dt

In [ ]:
train = dt.fread("/kaggle/input/tabular-playground-series-oct-2021/train.csv").to_pandas()
test = dt.fread("/kaggle/input/tabular-playground-series-oct-2021/test.csv").to_pandas()

In [ ]:
memory_usage = train.memory_usage(deep=True) / 1024 ** 2

memory_usage.sum()

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
unique_values = train.iloc[:1000].nunique()
categoricals = [col for col in  unique_values.index[unique_values < 10] if col!='target']
numeric = [col for col in test.columns if col not in categoricals]

train['mean_numeric'] = train[numeric].mean(axis=1)
train['std_numeric'] = train[numeric].std(axis=1)
train['min_numeric'] = train[numeric].min(axis=1)
train['max_numeric'] = train[numeric].max(axis=1)
train['sum_categoricals'] = train[categoricals].sum(axis=1)

test['mean_numeric'] = test[numeric].mean(axis=1)
test['std_numeric'] = test[numeric].std(axis=1)
test['min_numeric'] = test[numeric].min(axis=1)
test['max_numeric'] = test[numeric].max(axis=1)
test['sum_categoricals'] = test[categoricals].sum(axis=1)


In [ ]:

# # Sampling for speeding up things
# X = train.sample(n=400000, random_state=0)
# y = X['target']

In [ ]:
# X = X.drop(['id','target'],axis = 1)

# test_data = test.drop('id',axis = 1)

In [ ]:
# for submission
y = train.target
X = train.drop(['id','target'],axis = 1)

test_data = test.drop('id',axis = 1)

In [ ]:
train_df = reduce_memory_usage(X, verbose=True)
test_df = reduce_memory_usage(test_data, verbose=True)

In [ ]:
del train , test , X , test_data , memory_usage
gc.collect()

In [ ]:
# train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
# test = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")
# submission = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv")

In [ ]:
# %%time
# def objective(trial):
#     X_train,X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.35, shuffle=True , stratify = y)
    
#     # Parameters
#     params = {
        
#         "loss":trial.suggest_categorical("loss",["binary_crossentropy"]),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01,0.3),
#         "max_iter" : trial.suggest_int("max_iter", 50,2500),
#         "max_leaf_nodes":trial.suggest_int("max_leaf_nodes",2,200),
#         "min_samples_leaf" : trial.suggest_int("min_samples_leaf", 2,500),
#         "l2_regularization": trial.suggest_float("l2_regularization", 0.0,0.5),
#         "max_bins":trial.suggest_int("max_bins",2,255)
#         }
#     model = HistGradientBoostingClassifier( **params) 
        
#     model.fit(X_train, y_train)

#     # # Predict
#     preds = model.predict_proba(X_test)[:,1]
#     # # eval
#     roc_score = roc_auc_score(y_test,preds)
 
#     return roc_score

In [ ]:
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=200)

In [ ]:
# hg_params = {
#     'loss': 'binary_crossentropy',
#     'learning_rate': 0.11252047021457195,
#     'max_iter': 828, 
#     'max_leaf_nodes': 7,
#     'min_samples_leaf': 165,
#     'l2_regularization':0.01565259604437387,
#     'max_bins': 171
# }


hg_params = {
    'loss': 'binary_crossentropy',
    'learning_rate':0.12342440743956311,
    'max_iter': 1211, 
    'max_leaf_nodes': 6,
    'min_samples_leaf': 263,
    'l2_regularization':0.43176493333205407,
    'max_bins': 126
}


In [ ]:
folds = StratifiedKFold(n_splits = 5, random_state = 228, shuffle = True)
predictions_hg = np.zeros(len(test_df))
auc_score_hg = 0
# overall_auc_hg = 0

for fold, (trn_idx, val_idx) in enumerate(folds.split(train_df, y)):
    
    X_train, X_val = train_df.iloc[trn_idx], train_df.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]

    clf = HistGradientBoostingClassifier(**hg_params)
   
    clf.fit(X_train, y_train)
    preds = clf.predict_proba(X_val)[:,1]
   
    auc_score_hg = roc_auc_score(y_val, preds)
    print(f"The AUC of {fold + 1} fold is {auc_score_hg}")
#     overall_auc_hg += auc_score_hg
    del auc_score_hg 
    gc.collect()
    predictions_hg += clf.predict_proba(test_df)[:,1] / folds.n_splits 
    
    
# print(f"The over all AUC is {overall_auc_hg / 5}")

In [ ]:
submission = dt.fread("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv").to_pandas()

In [ ]:
submission['target'] = predictions_hg
submission.to_csv('hg_params_all.csv', index=False)

In [ ]:
submission